## 네이버 뉴스 sub 페이지 크롤링
- 한개의 sub 페이지 크롤링한 후 함수로 재구성
- 02_1번에서 수집된 데이터를 이용해서 여러 subpage크롤링

In [8]:
import requests
from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen


In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [10]:
# 뉴스 sub 페이지 문서 코드 받아오기

headers = {'User-Agent':
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
url = "https://news.naver.com/main/read.naver?mode=LPOD&mid=sec&oid=001&aid=0012576808"
result = requests.get(url, headers=headers)


In [11]:
result

<Response [200]>

In [22]:
#result.content # 전송된 소스코드를 반환
#result.text # 전송된 소스코드를 문자열로 반환

In [12]:
result.content

b'<!DOCTYPE HTML> \r\n<html lang="ko"> \r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="referrer" contents="always">\r\n<meta name="viewport" content="width=1106" />\r\n<title>\xb3\xbb\xb3\xe2\xb5\xb5 \xb9\xe9\xbd\xc5\xb5\xb5\xc0\xd4 \xc7\xf9\xbb\xf3 \xb8\xb6\xb9\xab\xb8\xae \xb4\xdc\xb0\xe8\xa1\xa6\xc8\xad\xc0\xcc\xc0\xda\xa1\xa4\xb8\xf0\xb4\xf5\xb3\xaa 5\xc3\xb5\xb8\xb8\xc8\xb8\xba\xd0(\xc1\xbe\xc7\xd5) : \xb3\xd7\xc0\xcc\xb9\xf6 \xb4\xba\xbd\xba</title>\r\n\n\n\t\n\t\n\t\t\n\t\n\n\n\t\n\t\n\t\n\t\t\n\t\t\n\t\t\n\t\n\n<meta property="me2:post_tag"\t\tcontent="\xbf\xac\xc7\xd5\xb4\xba\xbd\xba : \xb3\xd7\xc0\xcc\xb9\xf6\xb4\xba\xbd\xba"/>\n<meta property="me2:category1"\t\tcontent="\xbf\xac\xc7\xd5\xb4\xba\xbd\xba"/>\n<meta property="me2:category2"\t\tcontent="\xbc\xd3\xba\xb8"/>\n<meta property="me2:image"\t\t\tcontent="https://imgnews.pstatic.net/image/001/2021/08/04/PYH2021073007380001300_P4_20210804121119022.jpg"/>\n\n<me

In [6]:
result.text

'<!DOCTYPE HTML> \r\n<html lang="ko"> \r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="referrer" contents="always">\r\n<meta name="viewport" content="width=1106" />\r\n<title>내년도 백신도입 협상 마무리 단계…화이자·모더나 5천만회분(종합) : 네이버 뉴스</title>\r\n\n\n\t\n\t\n\t\t\n\t\n\n\n\t\n\t\n\t\n\t\t\n\t\t\n\t\t\n\t\n\n<meta property="me2:post_tag"\t\tcontent="연합뉴스 : 네이버뉴스"/>\n<meta property="me2:category1"\t\tcontent="연합뉴스"/>\n<meta property="me2:category2"\t\tcontent="속보"/>\n<meta property="me2:image"\t\t\tcontent="https://imgnews.pstatic.net/image/001/2021/08/04/PYH2021073007380001300_P4_20210804121119022.jpg"/>\n\n<meta property="og:title"\t\t\tcontent="내년도 백신도입 협상 마무리 단계…화이자·모더나 5천만회분(종합)"/>\n<meta property="og:type"\t\t\tcontent="article"/>\n<meta property="og:url"\t\t\t\tcontent="https://news.naver.com/main/read.naver?mode=LSD&mid=sec&oid=001&aid=0012576808&sid1=001"/>\n<meta property="og:image"\t\t\tcontent="https://imgnews.pstatic.net/image/00

In [13]:
bs_obj = BeautifulSoup(result.content,'html.parser')
#기사 제목 추출
title = bs_obj.find(id='articleTitle')
title
date_time = bs_obj.find('span', {'class':'t11'})
date_time

<h3 id="articleTitle">내년도 백신도입 협상 마무리 단계…화이자·모더나 5천만회분(종합)</h3>

<span class="t11">2021.08.04. 오후 12:10</span>

## 개발자 도구의 copy selector 사용법
- 크롤링하려는 컨텐츠에서 검사 -> 해당 주소에서 단축 메뉴 -> copy -> copy selector
    - 붙여넣기하면 선택자를 반환해주는데 선택된 내용이 없는 경우가 있음
        - 잘못된 선택자를 생성했을 경우 - 선택자 재구성
        - 네이버 뉴스 서브페이지의 좋아요 수와 기사 댓글 수는 동적 생성되어서 정적 크롤링 불가능
        - 셀레니움이라는 패키지를 사용해야 함

In [14]:
# sub 페이지에서 수집된 데이터를 반환하는 함수 작성 (서브페이지 url을 파라미터로 전달받음)

def get_news_sub_info(url):
    
    #1. 전달된 url에 접속 후 응답객체 반환
    headers = {'User-Agent':
               "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url, headers=headers)
    
    #2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, 'html.parser')
    
    #3. 필요 데이터 추출
    # 기사 제목 추출(태그객체)
    title = bs_obj.find(id='articleTitle')
    date_time = bs_obj.find('span', {'class':'t11'})
    
    #4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub['기사제목'] = title.text
    dict_sub['기사 입력 시간'] = date_time.text
    
    return dict_sub
    
    

In [ ]:
## main : 함수 호출 코드


In [15]:
headers = {'User-Agent':
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
url = "https://news.naver.com"

res = requests.get(url, headers=headers)

html = res.text

bs_obj = BeautifulSoup(html, 'html.parser')

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul", {"class": "hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")

link_list=[]

# 뉴스 제목 출력 : 
for li in lis :
    a_tag = li.find("a")
    link_list.append('https://news.naver.com' + a_tag['href'])

# link_list
for url in link_list:
    res = get_news_sub_info(url)
    print(res)

{'기사제목': "'슬의생2' 효과에 광고매출↑…CJ ENM, 코로나 불황 없었다", '기사 입력 시간': '2021.08.05. 오후 5:12'}
{'기사제목': '이번 수능 백신 미접종자도 응시 가능…가림막 재질 바꿔', '기사 입력 시간': '2021.08.05. 오전 11:38'}
{'기사제목': "송영길 '불임정당' 발언에 국민의힘·정의당 사과 요구", '기사 입력 시간': '2021.08.05. 오후 5:34'}
{'기사제목': '18~49세 예방접종 예약시스템 개선…시간당 200만명까지 처리', '기사 입력 시간': '2021.08.05. 오후 2:12'}
{'기사제목': '자매교회發 감염 100명 넘어…대구 121명 신규확진, 올들어 최다', '기사 입력 시간': '2021.08.05. 오후 2:20'}


## 연습문제

In [16]:
# sub 페이지에서 수집된 데이터를 반환하는 함수 작성 (서브페이지 url을 파라미터로 전달받음)

def get_news_sub_info(url):
    
    #1. 전달된 url에 접속 후 응답객체 반환
    headers = {'User-Agent':
               "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}
    result = requests.get(url, headers=headers)
    
    #2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, 'html.parser')
    
    #3. 필요 데이터 추출
    # 기사 제목 추출(태그객체)
    title = bs_obj.find(id='articleTitle')
    date_time = bs_obj.find('span', {'class':'t11'})
    
    # 기사 내용 추출(id=articleBodyContents)
    contents = bs_obj.find(id='articleBodyContents')
    
    
    #4. 수집 데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub['기사제목'] = title.text
    dict_sub['기사 입력 시간'] = date_time.text
    dict_sub['기사'] = contents
    
    return dict_sub
    
    

In [ ]:
## 함수 끝

In [ ]:
##  main 시작

In [17]:
## 빈 데이터 프레임 하나 생성
import pandas as pd

dict_sub = {} # 빈 dict 생성
dict_sub['기사제목'] = [] # 딕셔너리에 key 값 생성
dict_sub['기사 입력 시간'] = []
dict_sub['기사'] = []

news_sub = pd.DataFrame(dict_sub)

news_sub

,기사제목,기사 입력 시간,기사


In [18]:
headers = {'User-Agent':
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com"

res = requests.get(url, headers=headers)

html = res.text

bs_obj = BeautifulSoup(html, 'html.parser')

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul", {"class": "hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")

link_list=[]

# 뉴스 제목 출력 : 
for li in lis :
    a_tag = li.find("a")
    link_list.append('https://news.naver.com' + a_tag['href'])

# link_list
for url in link_list:
    res = get_news_sub_info(url)
    
    # 빈 df에 반환되는 dict 행으로 추가 (append 함수 이용)
    news_sub = news_sub.append(res,ignore_index=True)  # 리스트의 append와 다른 개념이라 원본을 반영되지 않았어 저렇게 표현

news_sub

,기사제목,기사 입력 시간,기사
0,"식중독 김밥집은 '마녀김밥'…""두렵지만 숨지않겠다"" 사과",2021.08.05. 오후 4:46,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
1,"""이제 백신예약 밤샘 안해도 됩니다""...서버확충·10부제로 먹통차단",2021.08.05. 오후 6:00,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
2,"인권위원장에 文동기 송두환, 이재명 무죄 만든 그 변호인",2021.08.05. 오후 3:54,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
3,'탄소 중립' 없는 탄소 중립? 탄중위의 3가지 시나리오,2021.08.05. 오후 5:21,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
4,"""'알짜 콘텐츠' 효과"" CJ ENM, 2Q 영업익 전년比 17%↑(상보)",2021.08.05. 오후 4:39,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."


## 기존에 수집한 데이터를 이용한 서브페이지 수집

In [19]:
news_df = pd.read_csv('./crawl_data/naver_news.csv')

In [20]:
news_df.head()

,Unnamed: 0,section,new_title,link_url
0,0,헤드라인,"홍남기 ""국민지원금 9월말 90%집행…채무연장·이자유예 검토중""(종합)",https://news.naver.com/main/read.naver?mode=LS...
1,1,헤드라인,"집회 제한 어긴 양경수 소환… “정부, 방역실패 책임 전가”",https://news.naver.com/main/read.naver?mode=LS...
2,2,헤드라인,톱4 없는 국힘 '쪽방촌 봉사'…대선버스 출발부터 삐걱,https://news.naver.com/main/read.naver?mode=LS...
3,3,헤드라인,"""SK이노베이션, 지주사 할인 적용해 투자의견·목표가↓""-하나금투",https://news.naver.com/main/read.naver?mode=LS...
4,4,헤드라인,‘법률 플랫폼’ 갈등 본격화?…변협 ‘로톡 변호사’ 징계 검토,https://news.naver.com/main/read.naver?mode=LS...


In [21]:
## 빈 데이터 프레임 생성 코드
import pandas as pd

dict_sub = {} # 빈 dict 생성
dict_sub['기사제목'] = [] # 딕셔너리에 key 값 생성
dict_sub['기사 입력 시간'] = []
dict_sub['기사'] = []

news_sub_main = pd.DataFrame(dict_sub)

news_sub_main

,기사제목,기사 입력 시간,기사


In [27]:
import time

link_list = news_df['link_url']


for url in link_list:
    res = get_news_sub_info(url)
    
    # 빈 df에 반환되는 dict 행으로 추가 (append 함수 이용)
    news_sub_main = news_sub_main.append(res,ignore_index=True)  # 리스트의 append와 다른 개념이라 원본을 반영되지 않았어 저렇게 표현
    
    time.sleep(2) ## 반복적인 요청으로 서버 disconnect 현상을 막기 위한 방법
    
news_sub_main

,기사제목,기사 입력 시간,기사
0,"홍남기 ""국민지원금 9월말 90%집행…채무연장·이자유예 검토중""(종합)",2021.08.05. 오전 8:15,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
1,"집회 제한 어긴 양경수 소환… “정부, 방역실패 책임 전가”",2021.08.05. 오전 5:07,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
2,톱4 없는 국힘 '쪽방촌 봉사'…대선버스 출발부터 삐걱,2021.08.05. 오전 8:39,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
3,"""SK이노베이션, 지주사 할인 적용해 투자의견·목표가↓""-하나금투",2021.08.05. 오전 7:48,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
4,‘법률 플랫폼’ 갈등 본격화?…변협 ‘로톡 변호사’ 징계 검토,2021.08.05. 오전 6:47,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
5,최재형 “이명박·박근혜 사면해야…국민대통합 차원”,2021.08.05. 오전 9:39,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
6,"""진중권, 변호사들이 가장 좋아하는 타입의 피고소인""",2021.08.05. 오전 9:32,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
7,"[최강시사] 이준석 “안철수 ‘전범’ 운운, 정상적인 정치 지도자 간 대화 아냐”",2021.08.05. 오전 9:30,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
8,"사흘간 영남 찾는 최재형, 월성 원전도 방문…윤석열, 나흘간 휴가",2021.08.05. 오전 9:27,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
9,"김근식 “이재명, 음주운전 공격받자 김연경 끌어들여”",2021.08.05. 오전 9:26,"[\n, 본문 내용 , \n, TV플레이어 , \n, // TV플레이어 , \..."
